In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
sz = 28 #image size
num_classes = 10

In [4]:
#Define placeholders
x = tf.placeholder(tf.float32, shape = [None, sz * sz], name = 'x') #image data
y_ = tf.placeholder(tf.float32, shape = [None, num_classes], name = 'y_') #correct labels

In [5]:
#Define functions that initialize the weights and biases
def weight_variable(name, shape):
    W_init = tf.truncated_normal(shape, stddev = 0.1)
    W = tf.Variable(W_init, name = 'W_' + name)
    return W

def bias_variable(name, size):
    b_init = tf.constant(0.1, shape = [size])
    b = tf.Variable(b_init, name = 'b_' + name)
    return b

In [6]:
#Define functions for conv2d and max_pool
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')

In [7]:
#conv1
#(28, 28, 1) -> (28, 28, 32)
with tf.name_scope('conv1') as scope:
    W_conv1 = weight_variable('conv1', [5, 5, 1, 32])
    b_conv1 = bias_variable('conv1', 32)
    x_image = tf.reshape(x, [-1, sz, sz, 1])
    a_conv1= tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [8]:
#pool1
#(28, 28, 32) -> (14, 14, 32)
with tf.name_scope('pool1') as scope:
    a_pool1 = max_pool(a_conv1)

In [9]:
#conv2
#(14, 14, 32) -> (14, 14, 64)
with tf.name_scope('conv2') as scope:
    W_conv2 = weight_variable('conv2', [5, 5, 32, 64])
    b_conv2 = bias_variable('conv2', 64)
    a_conv2 = tf.nn.relu(conv2d(a_pool1, W_conv2) + b_conv2)

In [10]:
#pool2
#(14, 14, 64) -> (7, 7, 64)
with tf.name_scope('pool2') as scope:
    a_pool2 = max_pool(a_conv2)

In [11]:
#fc1
with tf.name_scope('fc1') as scope:
    n = 7 * 7* 64
    W_fc1 = weight_variable('fc1', [n, 1024])
    b_fc1 = bias_variable('fc1', 1024)
    a_pool2_flatten = tf.reshape(a_pool2, [-1,n])
    a_fc1 = tf.nn.relu(tf.matmul(a_pool2_flatten, W_fc1) + b_fc1)

In [12]:
#Dropout
with tf.name_scope('dropout') as scope:
    keep_prob = tf.placeholder(tf.float32)
    a_fc1_dropout = tf.nn.dropout(a_fc1, keep_prob)

In [13]:
#fc2
with tf.name_scope('fc2') as scope:
    W_fc2 = weight_variable('fc2', [1024, 10])
    b_fc2 = bias_variable('fc2', 10)
    y_hat = tf.nn.softmax(tf.matmul(a_fc1_dropout, W_fc2) + b_fc2)

In [15]:
#Train
with tf.name_scope('loss') as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y_hat))
with tf.name_scope('training') as scope:
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(cross_entropy)

In [16]:
#Evaluate
with tf.name_scope('predict') as scope:
    predict_step = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_, 1))
    accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))

In [17]:
#Set up feed_dict
def set_feed(images, labels, probs):
    return {x : images, y_ : labels, keep_prob : probs}

In [18]:
#Start session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #Make a tendorboard
    file_writer = tf.summary.FileWriter('./log_dir', sess.graph)
    
    #Prepare feed_dict for validation set
    val_fd =set_feed(mnist.validation.images, mnist.validation.labels, 1) #Where 1 is the probability for the dropout
    
    #Prepare feed_dict for test set
    test_fd =set_feed(mnist.test.images, mnist.test.labels, 1) #Where 1 is the probability for the dropout
    #Training
    for step in range(2000):
        batch = mnist.train.next_batch(64)
        fd = set_feed(batch[0], batch[1], 0.5)
        _, train_loss = sess.run([train_step, cross_entropy], feed_dict = fd)
        if step % 100 == 0:
            val_loss, acc = sess.run([cross_entropy, accuracy_step], feed_dict = val_fd)
            #acc = sess.run(accuracy_step, feed_dict = test_fd)
            print("(step:", step,")  (train loss:", train_loss, ")  (val loss:", val_loss, ")  (val acc:", acc, ")")
    acc = sess.run(accuracy_step, feed_dict = test_fd)
    print("test acc:", acc)

step: 0 train loss 520.51685 val loss 28225.031 acc 0.1086
step: 100 train loss 88.31464 val loss 2515.5347 acc 0.8454
step: 200 train loss 41.37371 val loss 1608.0947 acc 0.9098
step: 300 train loss 16.521648 val loss 1253.5862 acc 0.9316
step: 400 train loss 29.039562 val loss 1055.1381 acc 0.9396
step: 500 train loss 26.068958 val loss 881.5731 acc 0.9518
step: 600 train loss 17.141338 val loss 783.6948 acc 0.9566
step: 700 train loss 5.507494 val loss 704.11755 acc 0.9588
step: 800 train loss 17.794321 val loss 668.49567 acc 0.9626
step: 900 train loss 12.510027 val loss 612.17993 acc 0.9646
step: 1000 train loss 8.853877 val loss 567.9082 acc 0.9674
step: 1100 train loss 12.885713 val loss 538.7213 acc 0.968
step: 1200 train loss 12.407662 val loss 503.77203 acc 0.9692
step: 1300 train loss 6.9274096 val loss 482.78033 acc 0.9708
step: 1400 train loss 5.9865294 val loss 482.42798 acc 0.9724
step: 1500 train loss 4.0209045 val loss 436.42532 acc 0.977
step: 1600 train loss 18.63519